# Consultas MONGODB

Nicolás Fernández y Néstor Villa

In [1]:
# Importamos las clases y el método init_app
from models import Cliente, Producto, Compra, Proveedor, init_app
from bson import ObjectId
from datetime import datetime

# Iniciamos la conexión a la base de datos
init_app()

In [2]:
#1. Listado de todas las compras de un cliente
pipeline = [
    {
        "$group": {
            "_id": "$cliente.nombre",
            "compras": { "$push": { "$toString": "$_id" } }
        }
    },
    {
        "$project": {
            "nombre_cliente": "$_id",
            "compras": 1,
            "_id": 0
        }
    }
]

# Ejecuta el pipeline con raw=True
compras_cliente = list(Compra.aggregate(pipeline, raw=True))

compras_cliente

[{'compras': ['671ead0e8c77f9e3209c78bc',
   '671ead0f8c77f9e3209c78cb',
   '671ead108c77f9e3209c78cc'],
  'nombre_cliente': 'Ana Sánchez'},
 {'compras': ['671ead0e8c77f9e3209c78b9',
   '671ead0e8c77f9e3209c78bb',
   '671ead0f8c77f9e3209c78c3',
   '671ead0f8c77f9e3209c78ca',
   '671ead108c77f9e3209c78d1'],
  'nombre_cliente': 'María Fernández'},
 {'compras': ['671ead0e8c77f9e3209c78ba',
   '671ead0f8c77f9e3209c78bd',
   '671ead0f8c77f9e3209c78c4',
   '671ead0f8c77f9e3209c78c9',
   '671ead108c77f9e3209c78cf',
   '671ead108c77f9e3209c78d0'],
  'nombre_cliente': 'Luis Martínez'},
 {'compras': ['671ead0f8c77f9e3209c78be',
   '671ead0f8c77f9e3209c78bf',
   '671ead0f8c77f9e3209c78c5',
   '671ead108c77f9e3209c78cd',
   '671ead108c77f9e3209c78ce',
   '671ead108c77f9e3209c78d2',
   '671ead108c77f9e3209c78d3',
   '671ead108c77f9e3209c78d4',
   '671ead108c77f9e3209c78d5',
   '671ead118c77f9e3209c78d6'],
  'nombre_cliente': 'Beatriz Gómez'},
 {'compras': ['671ead0f8c77f9e3209c78c0',
   '671ead0f8c

In [3]:
# 2. Listado de todos los proveedores para un producto
pipeline = [
    { "$unwind": "$proveedores" },
    {
        "$group": {
            "_id": "$nombre",
            "proveedores": { "$push": "$proveedores.nombre" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "nombre_producto": "$_id",
            "proveedores": 1
        }
    }
]

# Ejecuta el pipeline con raw=True
proveedores_producto = list(Producto.aggregate(pipeline, raw=True))

proveedores_producto

[{'proveedores': ['Modas Paqui'],
  'nombre_producto': 'Camiseta de manga corta blanca'},
 {'proveedores': ['Proveedor Este', 'Distribuciones Norte'],
  'nombre_producto': 'Gafas de sol'},
 {'proveedores': ['Logística Sur', 'Proveedor Este'],
  'nombre_producto': 'Vestido de fiesta'},
 {'proveedores': ['Modas Paqui', 'Proveedor Central', 'Distribuciones Norte'],
  'nombre_producto': 'Zapatos de vestir'},
 {'proveedores': ['Proveedor Este'], 'nombre_producto': 'Bufanda de seda'},
 {'proveedores': ['Proveedor Central'],
  'nombre_producto': 'Zapatos deportivos'},
 {'proveedores': ['Modas Paqui', 'Logística Sur', 'Proveedor Este'],
  'nombre_producto': 'Camiseta de manga corta azul'},
 {'proveedores': ['Proveedor Central',
   'Proveedor Este',
   'Distribuciones Norte'],
  'nombre_producto': 'Pantalón vaquero'},
 {'proveedores': ['Modas Paqui', 'Logística Sur'],
  'nombre_producto': 'Calcetines de algodón'},
 {'proveedores': ['Distribuciones Norte'],
  'nombre_producto': 'Chaqueta de cuer

In [4]:
# 3. Listado de todos los productos diferentes comprados por un cliente
pipeline = [
    { "$unwind": "$productos" },
    {
        "$group": {
            "_id": "$cliente.nombre",
            "productos": { "$addToSet": "$productos.nombre" }
        }
    },
    {
        "$project": {
            "nombre_cliente": "$_id",
            "productos": 1,
            "_id": 0
        }
    }
]

# Ejecuta el pipeline con raw=True
productos_cliente = list(Compra.aggregate(pipeline, raw=True))

# Muestra los resultados
productos_cliente    

[{'productos': ['Guantes de invierno',
   'Pantalón vaquero',
   'Camiseta de manga corta negra',
   'Camiseta de manga corta azul',
   'Gafas de sol',
   'Zapatos de vestir',
   'Jersey de lana',
   'Polo de manga corta verde',
   'Falda plisada',
   'Cinturón de cuero',
   'Traje de baño',
   'Camiseta de manga corta roja',
   'Zapatos deportivos'],
  'nombre_cliente': 'María Fernández'},
 {'productos': ['Jersey de lana',
   'Traje de baño',
   'Sombrero de paja',
   'Camiseta de manga corta negra',
   'Falda plisada',
   'Camiseta de manga corta blanca',
   'Vestido de fiesta',
   'Gafas de sol',
   'Bufanda de seda',
   'Pantalón vaquero',
   'Zapatos deportivos',
   'Calcetines de algodón',
   'Camiseta de manga corta azul',
   'Chaqueta de cuero',
   'Camisa de manga larga'],
  'nombre_cliente': 'Luis Martínez'},
 {'productos': ['Bufanda de seda',
   'Pantalón vaquero',
   'Zapatos deportivos',
   'Camiseta de manga corta negra',
   'Falda plisada',
   'Guantes de invierno',
   '

In [5]:
# 4. Listado de productos vendidos por “Modas Paqui” cuyo nombre contenga “manga corta”
pipeline = [
    { "$unwind": "$proveedores" },
    {
        "$match": {
            "proveedores.nombre": "Modas Paqui",
            "nombre": { "$regex": "manga corta", "$options": "i" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "nombre_producto": "$nombre",
            "proveedor": "$proveedores.nombre"
        }
    }
]

# Ejecuta el pipeline con raw=True
productos_manga_corta = list(Producto.aggregate(pipeline, raw=True))

productos_manga_corta

[{'nombre_producto': 'Camiseta de manga corta azul',
  'proveedor': 'Modas Paqui'},
 {'nombre_producto': 'Camiseta de manga corta blanca',
  'proveedor': 'Modas Paqui'},
 {'nombre_producto': 'Camiseta de manga corta negra',
  'proveedor': 'Modas Paqui'},
 {'nombre_producto': 'Camiseta de manga corta roja',
  'proveedor': 'Modas Paqui'},
 {'nombre_producto': 'Polo de manga corta verde', 'proveedor': 'Modas Paqui'}]

In [6]:
# 5. Calcular el peso y volumen total de los productos comprados por un cliente un día determinado
cliente_nombre = "María Fernández"  # Reemplaza con el nombre real del cliente
fecha_especifica = datetime(2023, 11, 4)  # Reemplaza con la fecha específica

pipeline = [
    # Filtrar las compras por cliente y fecha
    {
        "$match": {
            "cliente.nombre": cliente_nombre,
            "fecha_compra": fecha_especifica
        }
    },
    # Descomponer el arreglo de productos
    {"$unwind": "$productos"},
    # Calcular el volumen de cada producto
    {
        "$addFields": {
            "productos.volumen": {
                "$multiply": [
                    "$productos.dimensiones.ancho",
                    "$productos.dimensiones.alto",
                    "$productos.dimensiones.profundidad"
                ]
            }
        }
    },
    # Sumar el peso y el volumen de todos los productos
    {
        "$group": {
            "_id": None,
            "peso_total": { "$sum": "$productos.peso" },
            "volumen_total": { "$sum": "$productos.volumen" }
        }
    },
    {"$project": {"_id": 0,}}
]


# Ejecuta el pipeline con raw=True
peso_volumen_cliente = list(Compra.aggregate(pipeline, raw=True))

peso_volumen_cliente

[{'peso_total': 18.52, 'volumen_total': 50868}]

In [7]:
# 6. Calcular el número medio de envíos por mes y almacén
pipeline = [
    # Descomponer el arreglo de productos
    {"$unwind": "$productos"},
    # Descomponer el arreglo de proveedores por producto
    {"$unwind": "$productos.proveedores"},
    # Descomponer el arreglo de direcciones de almacenes por proveedor
    {"$unwind": "$productos.proveedores.direcciones_almacenes"},
    # Extraer el mes y el año de la fecha de compra
    {
        "$addFields": {
            "mes": { "$month": "$fecha_compra" },
            "anio": { "$year": "$fecha_compra" },
            "almacen": "$productos.proveedores.direcciones_almacenes"
        }
    },
    # Agrupar por almacén, mes y año para contar los envíos
    {
        "$group": {
            "_id": {
                "almacen": "$almacen",
                "mes": "$mes",
                "anio": "$anio"
            },
            "envios": { "$sum": 1 }
        }
    },
    # Calcular el promedio de envíos por mes y almacén
    {
        "$group": {
            "_id": {
                "almacen": "$_id.almacen",
                "mes": "$_id.mes"
            },
            "media_envios": { "$avg": "$envios" }
        }
    },
    # Agrupar nuevamente por almacén para recopilar los meses y medias
    {
        "$group": {
            "_id": "$_id.almacen",
            "envios_por_mes": {
                "$push": {
                    "mes": "$_id.mes",
                    "media_envios": "$media_envios"
                }
            }
        }
    },
    # Ordenar los meses dentro de cada almacén
    {
        "$project": {
            "_id": 0,
            "almacen": "$_id",
            "envios_por_mes": {
                "$sortArray": {
                    "input": "$envios_por_mes",
                    "sortBy": { "mes": 1 }
                }
            }
        }
    },
    # Ordenar los almacenes por ciudad para mejor legibilidad (opcional)
    {"$sort": {"almacen.ciudad": 1}}
]

# Ejecuta el pipeline con raw=True
promedio_envios = list(Compra.aggregate(pipeline, raw=True))

promedio_envios

[{'almacen': {'calle': 'Rua Augusta',
   'numero': '100',
   'ciudad': 'Lisboa',
   'codigo_postal': '1100-053',
   'pais': 'Portugal',
   'location': {'type': 'Point', 'coordinates': [-9.137398, 38.710141]}},
  'envios_por_mes': [{'mes': 1, 'media_envios': 2.0},
   {'mes': 2, 'media_envios': 12.0},
   {'mes': 3, 'media_envios': 2.0},
   {'mes': 4, 'media_envios': 6.0},
   {'mes': 5, 'media_envios': 2.0},
   {'mes': 7, 'media_envios': 4.0},
   {'mes': 8, 'media_envios': 1.0},
   {'mes': 9, 'media_envios': 4.0},
   {'mes': 11, 'media_envios': 3.0},
   {'mes': 12, 'media_envios': 4.0}]},
 {'almacen': {'calle': 'Calle de la Moda',
   'numero': '123',
   'ciudad': 'Madrid',
   'codigo_postal': '28015',
   'pais': 'España',
   'location': {'type': 'Point', 'coordinates': [-3.70379, 40.416775]}},
  'envios_por_mes': [{'mes': 1, 'media_envios': 3.0},
   {'mes': 2, 'media_envios': 9.0},
   {'mes': 3, 'media_envios': 5.0},
   {'mes': 4, 'media_envios': 10.0},
   {'mes': 5, 'media_envios': 4.0},

In [8]:
# 7. Listado con los tres proveedores con más volumen de facturación
pipeline = [
    # Descomponer el arreglo de productos
    {"$unwind": "$productos"},
    # Descomponer el arreglo de proveedores por producto
    {"$unwind": "$productos.proveedores"},
    # Agrupar por proveedor y sumar el precio de los productos
    {
        "$group": {
            "_id": "$productos.proveedores.nombre",
            "total_facturacion": { "$sum": "$productos.precio" }
        }
    },
    # Ordenar por total_facturacion descendente
    {"$sort": { "total_facturacion": -1 }},
    # Limitar a los tres primeros
    {"$limit": 3}
]


# Ejecuta el pipeline con raw=True
top_proveedores = list(Compra.aggregate(pipeline, raw=True))

top_proveedores

[{'_id': 'Logística Sur', 'total_facturacion': 5839.69},
 {'_id': 'Proveedor Este', 'total_facturacion': 5414.24},
 {'_id': 'Modas Paqui', 'total_facturacion': 5044.45}]

In [9]:
# 8. Listado de almacenes cerca de unas coordenadas determinadas (100km de distancia máxima)
pipeline = [
    {
        "$geoNear": {
            "near": { "type": "Point", "coordinates": [-9.537398, 38.710141] },
            "distanceField": "distancia",
            "maxDistance": 100 * 1000,
            "spherical": True,
            "query": { "direcciones_almacenes.location": { "$exists": True } }
        }
    },
    { "$unwind": "$direcciones_almacenes" },
    { "$sort": { "distancia": 1 } },
    {
        "$project": {
            "_id": 0,
            "calle_almacen": "$direcciones_almacenes.calle",
            "ciudad": "$direcciones_almacenes.ciudad",
            "distancia": 1,
            "location": "$direcciones_almacenes.location"
        }
    }
]

# Ejecuta el pipeline con raw=True
almacenes_cercanos = list(Proveedor.aggregate(pipeline, raw=True))

almacenes_cercanos

[{'distancia': 34745.68888511416,
  'calle_almacen': 'Rua Augusta',
  'ciudad': 'Lisboa',
  'location': {'type': 'Point', 'coordinates': [-9.137398, 38.710141]}}]

In [10]:
# 9. Listado de compras dentro de un polígono cuyos vértices están definidos
# Definir los vértices del polígono (longitud, latitud)
poligono = {
    "type": "Polygon",
    "coordinates": [[
        [-3.8880, 40.5687],
        [-3.5499, 40.5687],
        [-3.5499, 40.3120],
        [-3.8880, 40.3120],
        [-3.8880, 40.5687] # El polígono debe ser cerrado (el primer y último punto deben ser iguales)
    ]]
}

pipeline = [
    {
        "$match": {
            "direccion_envio.location": {
                "$geoWithin": {
                    "$geometry": poligono
                }
            }
        }
    },
    # Opcional: proyectar campos específicos
    {
        "$project": {
            "cliente.nombre": 1,
            "fecha_compra": 1,
            "direccion_envio": 1,
            "_id": 0
        }
    }
]


# Ejecuta el pipeline con raw=True
compras_poligono = list(Compra.aggregate(pipeline, raw=True))

compras_poligono

[{'cliente': {'nombre': 'Luis Martínez'},
  'fecha_compra': datetime.datetime(2024, 9, 29, 0, 0),
  'direccion_envio': {'calle': 'Calle Mayor',
   'ciudad': 'Madrid',
   'codigo_postal': '28013',
   'pais': 'España',
   'numero': '66',
   'location': {'type': 'Point', 'coordinates': [-3.710023, 40.415701]}}},
 {'cliente': {'nombre': 'Luis Martínez'},
  'fecha_compra': datetime.datetime(2023, 12, 18, 0, 0),
  'direccion_envio': {'calle': 'Calle Mayor',
   'ciudad': 'Madrid',
   'codigo_postal': '28013',
   'pais': 'España',
   'numero': '66',
   'location': {'type': 'Point', 'coordinates': [-3.710023, 40.415701]}}},
 {'cliente': {'nombre': 'Carlos López'},
  'fecha_compra': datetime.datetime(2024, 7, 28, 0, 0),
  'direccion_envio': {'calle': 'Gran Vía',
   'ciudad': 'Madrid',
   'codigo_postal': '28013',
   'pais': 'España',
   'numero': '63',
   'location': {'type': 'Point', 'coordinates': [-3.709754, 40.422474]}}},
 {'cliente': {'nombre': 'Carlos López'},
  'fecha_compra': datetime.da

In [11]:
# 10. Guardar en una tabla nueva el listado de compras que tienen que ser enviados desde un
#almacén en un día determinado
# Fecha específica
fecha_especifica = datetime(2023, 11, 4)
#Almacen
almacen_calle = "Calle de Alcalá"
almacen_ciudad = "Madrid"
almacen_pais = "España"
almacen_location = [-3.698789, 40.418102] 

pipeline = [
    # Filtrar las compras en la fecha específica
    {
        "$match": {
            "fecha_compra": fecha_especifica
        }
    },
    # Filtrar las compras que tienen algún producto que proviene del almacén especificado
    {
        "$match": {
            "productos.proveedores.direcciones_almacenes.location.coordinates": almacen_location
        }
    },
    # Agrupar las compras por fecha y almacén, recopilando los IDs de las compras
    {
        "$group": {
            "_id": {
                "fecha": "$fecha_compra",
                "almacen": {
                    # Dado que el almacén es específico, podemos utilizar los datos proporcionados
                    "calle": almacen_calle,
                    "ciudad": almacen_ciudad,
                    "pais": almacen_pais,
                    "location": {
                        "type": "Point",
                        "coordinates": almacen_location
                    }
                }
            },
            "compras": { "$addToSet": "$_id" }
        }
    },
    # Formatear el resultado
    {
        "$project": {
            "_id": 0,
            "fecha": "$_id.fecha",
            "almacen": "$_id.almacen",
            "compras": {
                "$map": {
                    "input": "$compras",
                    "as": "compra_id",
                    "in": { "compra_id": "$$compra_id" }
                }
            }
        }
    }
]

vista = list(Compra.aggregate(pipeline, raw=True))

vista

[{'fecha': datetime.datetime(2023, 11, 4, 0, 0),
  'almacen': {'calle': 'Calle de Alcalá',
   'ciudad': 'Madrid',
   'pais': 'España',
   'location': {'type': 'Point', 'coordinates': [-3.698789, 40.418102]}},
  'compras': [{'compra_id': ObjectId('671ead0e8c77f9e3209c78b9')}]}]